In [2]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft.client.api import NodeView

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /


✅ The installed version of syft==0.8.1b4 matches the requirement >=0.8.1b0 and the requirement <0.9


In [4]:
# Register a client to the domain
node = sy.orchestra.launch(name="test-domain-1", reset=True, dev_mode=True)
guest_domain_client = node.client
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

SQLite Store Path:
!open file:///var/folders/79/0t1vmtq92cb82c47x_vmxgj40000gn/T/7bca415d13ed4ec881f0d0aede098dbb.sqlite



/Users/ivolinengong/Documents/Research/OpenMined/PySyft/.venv/lib/python3.10/site-packages/IPython/core/formatters.py:367: FormatterWarning: text/html formatter returned invalid type <class 'type'> (expected <class 'str'>) for object: <SyftClient - test-domain-1 <7bca415d13ed4ec881f0d0aede098dbb>: PythonConnection>
  warnings.warn(


<SyftClient - test-domain-1 <7bca415d13ed4ec881f0d0aede098dbb>: PythonConnection>

In [5]:
# DS: Create function to submit to DO
@sy.syft_function(input_policy=sy.ExactMatch(),
                  output_policy=sy.SingleExecutionExactOutput())
def test_func():
    return 1
test_func()

1

In [6]:
# DS: Submit request to DO
submitted_req = guest_domain_client.api.services.code.request_code_execution(test_func)
submitted_req

```python
class Request:
  id: str = 19e9418755f942968c9af770a425e75d
  requesting_user_verify_key: str = 81484e73acad0fd278c2dcd5b587ca5040e949141d3f2dffe50c5f76b10f3620
  approving_user_verify_key: str = None
  request_time: str = 2023-06-02 17:14:35
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed4ec881f0d0aede098dbb
  request_hash: str = "f1e9fbdf485cf883e98e46875b0f7ea149c0b191792bc9699d1f0a9959611587"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

---

In [7]:
# DO: Login and veiw request
domain_client = node.login(email="info@openmined.org", password="changethis")
notification = domain_client.notifications[-1]
request = notification.link
request
# notification.link.changes[0].link

```python
class Request:
  id: str = 19e9418755f942968c9af770a425e75d
  requesting_user_verify_key: str = 81484e73acad0fd278c2dcd5b587ca5040e949141d3f2dffe50c5f76b10f3620
  approving_user_verify_key: str = None
  request_time: str = 2023-06-02 17:14:35
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed4ec881f0d0aede098dbb
  request_hash: str = "f1e9fbdf485cf883e98e46875b0f7ea149c0b191792bc9699d1f0a9959611587"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [8]:
# DO: Run code from DS, which maybe unsafe
user_code = request.changes[0].link
result = user_code.unsafe_function()
result

1

In [9]:
# DO: Approve request from DS
request.accept_by_depositing_result(result)  

<class 'syft.service.response.SyftSuccess'>: Request 19e9418755f942968c9af770a425e75d changes applied

---

In [10]:
# DS:  Gets pointer to result from DO
guest_domain_client._api = None
_ = guest_domain_client.api

result = guest_domain_client.api.services.code.test_func()
result

Pointer:
ActionDataEmpty UID: 6ee1c3518e0d4ec898c87077446a17fc <None>

In [11]:
type(result)

syft.service.action.action_object.AnyActionObject

In [15]:
# DS: Use get_from to request results
# guest_domain_client.api.services.action.get(result.id)
result.get_from(guest_domain_client)

1

1



Idea here was to modify the call functon in the user_code_service because when the data scientist request the results, this is the function that is called right?

But the call function returns the results, so we simply had to replace those results with an empty data object. So, unless the data scientist explicitly request for the results, he cant access them.

Next Steps

So, next steps will be to test this on all notebooks , debug if any errors and make sure it works.

<!-- Idea
Create an empty action object that just ensures that the returned


Just to be clear that I understand.

Our idea here was to modify the call functon in the user_code_service because when the data scientist request the results, this is the function that is called right?

But the call function returns the results, so we simply had to replace those results with an empty data object. So, unless the data scientist explicitly request for the results, he cant access them.


=======

Since it works now, I guess the next steps will be to ensure that it works on all notebooks and debug it.  -->